In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

C:\Users\hdadmin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path = 'D:\\PythonSamples\\Talking\\'

def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    #df['dow']      = df['datetime'].dt.dayofweek
    #df["doy"]      = df["datetime"].dt.dayofyear
    df["hod"]      = df["datetime"].dt.hour
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['click_time', 'datetime'], axis=1, inplace=True)
    return df

In [3]:
start_time = time.time()

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [4]:
train_ref = pd.read_csv(path+"train.csv", usecols=train_columns, dtype=dtypes)

In [6]:
test_ref = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)

In [7]:
# Drop the IP and the columns from target
train_ref.drop(['is_attributed'], axis=1, inplace=True)
gc.collect()
#nrow_train = train.shape[0]
merge_ref = pd.concat([train_ref, test_ref])
del train_ref, test_ref
gc.collect()

14

In [8]:
#ip_count = merge_ref.groupby(['ip'])['channel'].count().reset_index()
#ip_count.columns = ['ip', 'clicks_by_ip']

In [9]:
app_count = merge_ref.groupby(['app'])['channel'].count().reset_index()
app_count.columns = ['app', 'clicks_by_app']

In [10]:
del merge_ref
gc.collect()

21

In [11]:
train_check = pd.read_csv(path+"train.csv", usecols=train_columns, dtype=dtypes)

In [12]:
#train_check.groupby(['hod'])['is_attributed'].sum()

In [13]:
#train_check["is_attributed"].sum()

In [14]:
apprank = train_check.groupby(['app'])['is_attributed'].sum().reset_index().astype('int')
apprank.columns = ['app', 'app_sum']

In [15]:
rnk = apprank.loc[apprank['app_sum'] != 0]

In [16]:
#rnk.sort_values(by='app_sum', ascending=False)

In [17]:
rnk['app_rank'] = rnk['app_sum'].rank(ascending=False).astype('int')

C:\Users\hdadmin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
#%matplotlib inline
#plt.hist(rnk['app_sum'], normed=True, bins=100)

In [19]:
rnk.drop(['app_sum'], axis=1, inplace=True)

C:\Users\hdadmin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
#apprank.loc[apprank['app_sum'] != 0]

In [21]:
del apprank
gc.collect()

38

In [22]:
train_1 = train_check.loc[train_check["is_attributed"] == 1]

In [23]:
train_0 = train_check.loc[train_check["is_attributed"] == 0]

In [24]:
fraction_val = 500000/184447045
print(fraction_val)

0.0027108051527743367


In [25]:
train_0f = train_0.sample(frac=fraction_val)

In [26]:
train = pd.concat([train_1, train_0f])
del  train_0f, train_1
gc.collect()

0

In [27]:
train.count()

ip               956846
app              956846
device           956846
os               956846
channel          956846
click_time       956846
is_attributed    956846
dtype: int64

In [28]:
test = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)

In [29]:
# Drop the IP and the columns from target
y = train['is_attributed']
train.drop(['is_attributed'], axis=1, inplace=True)

# Drop IP and ID from test rows
sub = pd.DataFrame()
sub['click_id'] = test['click_id'].astype('int')
#test.drop(['click_id'], axis=1, inplace=True)
#gc.collect()

#nrow_train = train.shape[0]
#merge = pd.concat([train, test])
del test
gc.collect()

37

In [30]:
# Count the number of clicks by ip
#train = pd.merge(train, ip_count, on='ip', how='left', sort=False)
#train['clicks_by_ip'] = train['clicks_by_ip'].astype('uint16')

In [31]:
train = pd.merge(train, app_count, on=['app'], how='left', sort=False)
train['clicks_by_app'] = train['clicks_by_app'].astype('uint16')

In [32]:
train = pd.merge(train, rnk, on=['app'], how='left', sort=False)
train['app_rank'].fillna(0, inplace=True)

In [33]:
train['app_rank'] = train['app_rank'].astype('uint16')

In [34]:
train.drop('ip', axis=1, inplace=True)

In [35]:
gc.collect()

35

In [36]:
print('[{}] Start to generate time features'.format(time.time() - start_time))

train = timeFeatures(train)
gc.collect()

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[667.477912902832] Start to generate time features
[668.154007434845] Start XGBoost Training


In [37]:
def active_time(row):
    if row['hod'] in [17,18,19,20,21,22]:
        return 0
    else:
        return 1

In [38]:
train['active_time'] = train.apply(active_time, axis = 1)

In [39]:
train.drop('hod', axis=1, inplace=True)

In [40]:
train.corr()

,app,device,os,channel,clicks_by_app,app_rank,active_time
app,1.000000,0.093929,0.117683,-0.129948,-0.096558,0.310322,0.002489
device,0.093929,1.000000,0.532388,-0.011545,-0.040378,-0.027602,-0.004470
os,0.117683,0.532388,1.000000,-0.018491,-0.020147,0.000589,-0.000779
channel,-0.129948,-0.011545,-0.018491,1.000000,0.085558,0.045764,-0.026573
clicks_by_app,-0.096558,-0.040378,-0.020147,0.085558,1.000000,0.116712,-0.024487
app_rank,0.310322,-0.027602,0.000589,0.045764,0.116712,1.000000,-0.019893
active_time,0.002489,-0.004470,-0.000779,-0.026573,-0.024487,-0.019893,1.000000


In [41]:
#due high correlation between device and os, droping device coz it has high number of unique values.
#train.drop('device', axis=1, inplace=True)

In [42]:
#train.drop('', axis=1, inplace=True)

In [43]:
train.head()

,app,device,os,channel,clicks_by_app,app_rank,active_time
0,35,1,13,21,28753,2,1
1,9,1,22,215,62860,6,1
2,35,1,13,21,28753,2,1
3,35,1,46,274,28753,2,1
4,35,1,13,274,28753,2,1


In [44]:
print(xgb.__version__)

0.7


In [45]:
test = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)

In [46]:
#test = pd.merge(test, ip_count, on='ip', how='left', sort=False)
test = pd.merge(test, app_count, on=['app'], how='left', sort=False)
test = pd.merge(test, rnk, on=['app'], how='left', sort=False)

In [47]:
test['app_rank'].fillna(0, inplace=True)

In [48]:
#del ip_count
del app_count
del rnk
gc.collect()

204

In [49]:
#test['clicks_by_ip'] = test['clicks_by_ip'].astype('uint16')
test['clicks_by_app'] = test['clicks_by_app'].astype('uint16')
test['app_rank'] = test['app_rank'].astype('uint16')
test = timeFeatures(test)
test['active_time'] = test.apply(active_time, axis = 1)
test.drop(['click_id', 'ip'], axis=1, inplace=True)

In [50]:
test.drop('hod', axis=1, inplace=True)

In [48]:
#test.drop('device', axis=1, inplace=True)

In [95]:
#test.drop('active_time', axis=1, inplace=True)

In [51]:
test.head()

,app,device,os,channel,clicks_by_app,app_rank,active_time
0,9,1,3,107,62860,6,1
1,9,1,3,466,62860,6,1
2,21,1,19,128,60834,58,1
3,15,1,13,111,64259,17,1
4,12,1,13,328,8542,19,1


In [73]:
gc.collect()

18790989

In [53]:
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV

C:\Users\hdadmin\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [54]:
features = list(train.columns) 
print(features)

['app', 'device', 'os', 'channel', 'clicks_by_app', 'app_rank', 'active_time']


In [55]:
for f in train.columns:
    if train[f].dtype=='object':
        #print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))

In [56]:
xgb_model = xgb.XGBClassifier()

In [57]:
# Set the params(this params from Pranav kernel) for xgboost model
parameters = {'learning_rate': [0.5,0.6],
          'booster': ["gbtree"],
          'max_depth': [4,5],
          'subsample': [1.0],
          'colsample_bytree': [0.9,1.0],
          'min_child_weight':[0,1],
          'reg_alpha':[5.0,6.0],
          'gamma':[0.9],
          'objective': ['binary:logistic'], 
          'scale_pos_weight':[1,2],
          'nthread':[5],
          'random_state': [99], 
          'silent': [1]}

Raw AUC score: 0.9656840221349073
booster: 'gbtree'
colsample_bytree: 0.9
gamma: 0.8
learning_rate: 0.5
max_depth: 5
min_child_weight: 1
nthread: 5
objective: 'binary:logistic'
random_state: 99
reg_alpha: 5.0
scale_pos_weight: 2
silent: 1
subsample: 1.0

In [58]:
import datetime
datetime.datetime.now()

datetime.datetime(2018, 4, 8, 11, 39, 39, 900380)

In [59]:
clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y, n_folds=5, shuffle=True), 
                   scoring='roc_auc',
                   verbose=2, refit=True)

In [60]:
clf.fit(train[features], y)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:  8.5min
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed: 43.3min
[Parallel(n_jobs=5)]: Done 320 out of 320 | elapsed: 93.6min finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 1 ..., 0 0], n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'learning_rate': [0.4, 0.5], 'booster': ['gbtree'], 'max_depth': [4, 5], 'subsample': [1.0], 'colsample_bytree': [0.8, 0.9], 'min_child_weight': [0, 1], 'reg_alpha': [5.0, 6.0], 'gamma': [0.8], 'objective': ['binary:logistic'], 'scale_pos_weight': [1, 2], 'nthread': [5], 'random_state': [99], 'silent': [1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose

In [61]:
datetime.datetime.now()

datetime.datetime(2018, 4, 8, 13, 13, 59, 89378)

In [62]:
#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

Raw AUC score: 0.9656840221349073
booster: 'gbtree'
colsample_bytree: 0.9
gamma: 0.8
learning_rate: 0.5
max_depth: 5
min_child_weight: 1
nthread: 5
objective: 'binary:logistic'
random_state: 99
reg_alpha: 5.0
scale_pos_weight: 2
silent: 1
subsample: 1.0


In [63]:
test_probs = clf.predict_proba(test[features])[:,1]

In [64]:
sub['is_attributed'] = np.round(test_probs).astype(np.int64)

In [65]:
sub.head()

,click_id,is_attributed
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [66]:
sub.to_csv(path+'xgb_sub.csv',index=False)